In [ ]:
import pandas as pd
import re
import requests
from lxml import html
import sys

In [ ]:
class Scrapper:
    
    def __init__(self, source, link_pattern, xpath, xpathImg, df, max_while = 4, max_links = 20):
        self.source = source
        self.link_pattern = link_pattern
        self.xpath = xpath
        self.xpathImg = xpathImg
        self.df = df
        self.max_while = max_while
        self.max_links = max_links
        self.link_storage = []
        self.link_history = []
        
    def scrape(self, seed_link):
        self.link_storage.append(seed_link)
        i = 0
        while True:
            links_list_length = len(self.link_storage)
            links = self.link_storage.copy()
            for link in links:
                if link not in self.link_history:
                    seed_page = self.load(link)
                    self.link_history.append(link)
                    if seed_page:
                        if self.is_needed(link):
                            self.extract_info(seed_page)
                        self.extract_links(seed_page)
                if self.df.shape[0] > self.max_links:
                    break
            if links_list_length == len(self.link_storage):
                 i += 1
            if self.df.shape[0] > self.max_links or i > self.max_while:
                break
        return self.df
            
    def extract_links(self, page):
        pattern = self.source + '[a-z0-9\-\_\.\/]+'
        links = re.findall(pattern, page.text)
        for link in links:
            if not link in self.link_storage:
                self.link_storage.append(link)
    
    def load(self, link):
        response = requests.get(link)
        self.link_history.append(link)
        return response
    
    def is_needed(self, link):
        pattern = '[a-z0-9\-\_\.\/]+' + self.link_pattern
        return bool(re.search(pattern, link))
        
    def extract_info(self, page):
        tree = html.fromstring(page.content.decode('UTF-8'))
        head = tree.xpath(self.xpath)
        img_url = tree.xpath(self.xpathImg)
        #print(pic_url)
        data = [head, img_url]
        self.df = self.df.append(pd.DataFrame([data], columns=list(self.df.columns)), ignore_index=True)
        #print('did extract_info')
        #return res

In [ ]:
df = pd.DataFrame(columns = ['title', 'img_url'])

scrapper_kloop = Scrapper('https://www.vb.kg', '/doc/',
                           xpath = '//h1/text()',
                           xpathImg = '''(//div[@class="stk-mask"]/img/@src)[1]''',
                           df=df)

In [ ]:
scrapper_kloop.scrape('https://www.vb.kg/')

In [ ]:
scrapper_kloop.df